## Overview

This notebook shows you how to load data from JDBC databases using Spark SQL.

*For production, you should control the level of parallelism used to read data from the external database, using the parameters described in the documentation.*

### Step 1: Connection Information

This is a **Python** notebook so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` magic command. Python, Scala, SQL, and R are all supported.

First we'll define some variables to let us programmatically create these connections.

In [0]:
pip install json5

Python interpreter will be restarted.
Collecting json5
 Downloading json5-0.9.5-py2.py3-none-any.whl (17 kB)
Installing collected packages: json5
Successfully installed json5-0.9.5
Python interpreter will be restarted.

In [0]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://queenie.db.elephantsql.com:5432/ycfglres"
user = "ycfglres"
password = "z-PQKACVeHQ_f6wbuQQPpmyDJ55IhNwN"

### Step 2: Reading the data

Now that we specified our file metadata, we can create a DataFrame. You'll notice that we use an *option* to specify that we'd like to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python, notice how we will programmatically reference the variables we defined above.

In [0]:
import json5 as json

print("============== ITEMS FROM POSTGRES ==============")
table = "item"
df_items = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()

print("df_items")
df_items.show()
#df_items.toJSON().collect(): ['{"Id":1,"Name":"Bananas","Desc":"Bananas","Active":"1"}']
items = []
for e in df_items.toJSON().collect():
  e = json.loads(e)
  items.append(e)
print("Items as JSON Array", items)
bc_items = spark.sparkContext.broadcast(items)


print("============== STORE FROM POSTGRES ==============")
table = "store"
df_store = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()

print("df_store")
df_store.show()
stores = []
#['{"Id":1,"Name":"Store-A","City":"New York","State":"New York","Country":"USA","Active":1}']
for e in df_store.toJSON().collect():
  e = json.loads(e)
  stores.append(e)
print("Stores as JSON Array", stores)
bc_stores = spark.sparkContext.broadcast(stores)


print("============== TRANSACTIONS FROM POSTGRES ==============")
table = "transaction"
df_trans = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()
print("df_trans")
df_trans.show()


============== ITEMS FROM POSTGRES ==============
df_items
+---+-------+-------------+------+
 Id| Name| Desc|Active|
+---+-------+-------------+------+
 1|Bananas| Bananas| 1|
 2| Apples| Gala Apples| 1|
 3|Organes|Naval Oranges| 1|
+---+-------+-------------+------+

Items as JSON Array [{'Id': 1, 'Name': 'Bananas', 'Desc': 'Bananas', 'Active': '1'}, {'Id': 2, 'Name': 'Apples', 'Desc': 'Gala Apples', 'Active': '1'}, {'Id': 3, 'Name': 'Organes', 'Desc': 'Naval Oranges', 'Active': '1'}]
============== STORE FROM POSTGRES ==============
df_store
+---+-------+----------+--------+-------+------+
 Id| Name| City| State|Country|Active|
+---+-------+----------+--------+-------+------+
 1|Store-A| New York|New York| USA| 1|
 2|Store-B|Washington| Seattle| USA| 1|
 3|Store-C| Dallas| Texas| USA| 1|
+---+-------+----------+--------+-------+------+

Stores as JSON Array [{'Id': 1, 'Name': 'Store-A', 'City': 'New York', 'State': 'New York', 'Country': 'USA', 'Active': '1'}, {'Id': 2, 'Name': 'Store-B', 'City': 'Washington', 'State': 'Seattle', 'Country': 'USA', 'Active': '1'}, {'Id': 3, 'Name': 'Store-C', 'City': 'Dallas', 'State': 'Texas', 'Country': 'USA', 'Active': '1'}]
============== TRANSACTIONS FROM POSTGRES ==============
df_trans
+---+-------+------+--------------------+----------+
TId|StoreId|ItemId| TDesc| TDate|
+---+-------+------+--------------------+----------+
 1| 1| 2|Apples for StoreB...|2021-04-19|
 2| 3| 3|Oranges for Store...|2021-04-18|
 3| 2| 1|Bananas for Store...|2021-04-17|
+---+-------+------+--------------------+----------+

### Step 3: Querying the data

Now that we created our DataFrame. We can query it. For instance, you can select some particular columns to select and display within Databricks.

In [0]:
display(df_items.select("Name"))
display(df_store.select("Name"))
display(df_trans.select("TDesc"))

In [0]:
print("===================== MAP =====================")
import json5 as json
import pyspark.sql.functions as f
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg
from pyspark.sql.types import Row
import json5 as json

def myTransEnrich(row, bc_stores, bc_items):
  print("myTransEnrich::type:%s, data:%s" % (type(row), row))
  #myTransEnrich::type:<class 'pyspark.sql.types.Row'>, data:Row(TId=1, StoreId=3, ItemId=3, TDesc='Oranges for StoreC@DAL', TDate='2021-04-19T02:01:01')
  row_dict = row.asDict()
  print("myTransEnrich::row_dict:", type(row_dict), row_dict)
  #myTransEnrich::row_dict: <class 'dict'> {'TId': 1, 'StoreId': 2, 'ItemId': 1, 'TDesc': 'Bananas for StoreA@NYC', 'TDate': '2021-04-19T03:01:01'}
  itemId = row_dict.get("ItemId")
  storeId = row_dict.get("StoreId")
        
  #print("myTransEnrich::bc_user.value::", type(bc_stores.value), bc_stores.value)
  #myTransEnrich::bc_user.value:: <class 'list'> [{'Id': 1, 'Name': 'Store-A', 'City': 'New York', 'State': 'New York', 'Country': 'USA', 'Active': 1}]
  myitem = list(filter(lambda x:x["Id"]==storeId, bc_stores.value))
  print("myTransEnrich::myitem:", (myitem))
  #myTransEnrich::myitem: [{'Id': 3, 'Name': 'Store-C', 'City': 'Dallas', 'State': 'Texas', 'Country': 'USA', 'Active': 1}]
  myitem = myitem[0]
  row_dict["ItemName"] = myitem["Name"]

  #print("myTransEnrich::bc_items.value::", type(bc_items.value), bc_items.value)
  #myTransEnrich::bc_items.value:: <class 'list'> [{'Id': 1, 'Name': 'Bananas', 'Desc': 'Bananas', 'Active': '1'}]
  mystore = list(filter(lambda x:x["Id"]==itemId, bc_items.value))
  print("myTransEnrich::mystore:", (mystore))
  #myTransEnrich::mystore: [{'Id': 3, 'Name': 'Organes', 'Desc': 'Naval Oranges', 'Active': '1'}]
  mystore = mystore[0]
  row_dict["StoreName"] = mystore["Name"]
  
  return Row(**row_dict)

===================== MAP =====================

In [0]:
df_enriched = df_trans.rdd.map(lambda row: myTransEnrich(row, bc_stores, bc_items))
print("df_enriched:0:", df_enriched)
df_enriched.toDF(["TId", "StoreId", "ItemId", "TDesc", "TDate"]).show()

#remote_table.write.format("parquet").saveAsTable("MY_PERMANENT_TABLE_NAME")

df_enriched:0: PythonRDD[98] at RDD at PythonRDD.scala:58
+---+-------+------+--------------------+----------+--------+---------+
TId|StoreId|ItemId| TDesc| TDate|ItemName|StoreName|
+---+-------+------+--------------------+----------+--------+---------+
 1| 1| 2|Apples for StoreB...|2021-04-19| Store-A| Apples|
 2| 3| 3|Oranges for Store...|2021-04-18| Store-C| Organes|
 3| 2| 1|Bananas for Store...|2021-04-17| Store-B| Bananas|
+---+-------+------+--------------------+----------+--------+---------+

This table will persist across cluster restarts as well as allow various users across different notebooks to query this data. However, this will not connect back to the original database when doing so.